# Generate a TLE breakup

- use the EVOLVE model and break up a TLE (for now, at epoch)

In [ ]:
import sys
import sys

from sgp4.io import twoline2rv
from sgp4.propagation import sgp4 as sgprop
from sgp4.earth_gravity import wgs72

def addPath( P ):
    pth = os.path.abspath('..')
    if pth not in sys.path: sys.path.insert(0,pth)
        
from EVOLVE import evolve_collision

import numpy as np
import pandas as pd

In [ ]:
TLE = '1 32258U 07046A   25308.81572472 -.00000255  00000-0  00000-0 0  9994', '2 32258   0.1483  87.0630 0001016 141.6959  66.7150  1.00271503 50082'

tleo  = twoline2rv( *TLE, wgs72 )
tlejd = tleo.jdsatepoch
epoch = pd.to_datetime( tlejd, unit='D', origin='julian')
P,V   = [np.array(X) for X in sgprop( tleo, 0)]

WGS_mass    = 5987 # kg, from Gunther
WGS_charlen = 15   # meters; guess

print(epoch)
print('\n'.join(TLE))

In [ ]:
# model this as a non-catatrophic collision
non_catastrophic = evolve_collision( WGS_mass, 25, 
                     V, 0.95*V,
                     WGS_charlen,0.1,
                     min_characteristic_length=0.01 )
non_catastrophic.match_mass()
non_catastrophic.catastrophic

In [ ]:
epochstr = epoch.strftime('%Y %m %d %H %M %S.%f')

print('''DEFINE CLOCK
        EPOCH  {}
        TIME_STEP 60\n'''.format( epoch.strftime('%Y %m %d %H %M %S') ) )

for i,R in non_catastrophic.sim_output().iterrows():
    print( 'DEFINE PLATFORM KEPLER  "{:006d}"'.format(i) )
    delV = np.hstack( (R['vel_x'],R['vel_y'],R['vel_z'] ) ) / 1e3
    newV = delV + V
    print('\tSTATE "Earth"   CARTESIAN       CUSTOM {} {} {} {} {} {} {}'.format( *P, *newV , epochstr) )
    print('\tSHAPE DOT')


In [ ]:
non_catastrophic.sim_output()